## 准备数据

In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, optimizers, datasets

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # or any {'0', '1', '2'}

def mnist_dataset():
    (x, y), (x_test, y_test) = datasets.mnist.load_data()
    #normalize
    x = x/255.0
    x_test = x_test/255.0
    
    return (x, y), (x_test, y_test)

In [2]:
print(list(zip([1, 2, 3, 4], ['a', 'b', 'c', 'd'])))

[(1, 'a'), (2, 'b'), (3, 'c'), (4, 'd')]


## 建立模型

In [6]:
class myModel:
    def __init__(self):
        ####################
        '''声明模型对应的参数'''
        ####################
        self.W1 = tf.Variable(tf.random.normal([784, 256]), trainable=True)
        self.b1 = tf.Variable(tf.random.normal([256]), trainable=True)
        self.W2 = tf.Variable(tf.random.normal([256, 128]), trainable=True)
        self.b2 = tf.Variable(tf.random.normal([128]), trainable=True)
        self.W3 = tf.Variable(tf.random.normal([128, 10]), trainable=True)
        self.b3 = tf.Variable(tf.random.normal([10]), trainable=True)
        self.trainable_variables = [self.W1, self.b1, self.W2, self.b2, self.W3, self.b3]

    def __call__(self, x):
        ####################
        '''实现模型函数体，返回未归一化的logits'''
        ####################
        x = tf.reshape(x, [-1, 784])
        h1 = tf.nn.relu(x @ self.W1 + self.b1)
        h2 = tf.nn.relu(h1 @ self.W2 + self.b2)
        logits = h2 @ self.W3 + self.b3
        return logits
        
model = myModel()

optimizer = optimizers.Adam()

## 计算 loss

In [7]:
@tf.function
def compute_loss(logits, labels):
    return tf.reduce_mean(
        tf.nn.sparse_softmax_cross_entropy_with_logits(
            logits=logits, labels=labels))

@tf.function
def compute_accuracy(logits, labels):
    predictions = tf.argmax(logits, axis=1)
    return tf.reduce_mean(tf.cast(tf.equal(predictions, labels), tf.float32))

@tf.function
def train_one_step(model, optimizer, x, y):
    with tf.GradientTape() as tape:
        logits = model(x)
        loss = compute_loss(logits, y)

    # compute gradient
    trainable_vars = [model.W1, model.W2, model.b1, model.b2]
    grads = tape.gradient(loss, trainable_vars)
    for g, v in zip(grads, trainable_vars):
        v.assign_sub(0.01*g)

    accuracy = compute_accuracy(logits, y)

    # loss and accuracy is scalar tensor
    return loss, accuracy

@tf.function
def test(model, x, y):
    logits = model(x)
    loss = compute_loss(logits, y)
    accuracy = compute_accuracy(logits, y)
    return loss, accuracy

## 实际训练

In [8]:
train_data, test_data = mnist_dataset()
for epoch in range(50):
    loss, accuracy = train_one_step(model, optimizer, 
                                    tf.constant(train_data[0], dtype=tf.float32), 
                                    tf.constant(train_data[1], dtype=tf.int64))
    print('epoch', epoch, ': loss', loss.numpy(), '; accuracy', accuracy.numpy())
loss, accuracy = test(model, 
                      tf.constant(test_data[0], dtype=tf.float32), 
                      tf.constant(test_data[1], dtype=tf.int64))

print('test loss', loss.numpy(), '; accuracy', accuracy.numpy())

epoch 0 : loss 1198.4199 ; accuracy 0.06025
epoch 1 : loss 727.4111 ; accuracy 0.15238333
epoch 2 : loss 587.6943 ; accuracy 0.18146667
epoch 3 : loss 437.3824 ; accuracy 0.27423334
epoch 4 : loss 333.5428 ; accuracy 0.33973333
epoch 5 : loss 274.93164 ; accuracy 0.39145
epoch 6 : loss 247.21707 ; accuracy 0.42666668
epoch 7 : loss 227.10622 ; accuracy 0.45783332
epoch 8 : loss 211.56985 ; accuracy 0.4792
epoch 9 : loss 198.21573 ; accuracy 0.50123334
epoch 10 : loss 187.13916 ; accuracy 0.5178667
epoch 11 : loss 177.6266 ; accuracy 0.53506666
epoch 12 : loss 169.34087 ; accuracy 0.5484167
epoch 13 : loss 162.06047 ; accuracy 0.56161666
epoch 14 : loss 155.57683 ; accuracy 0.57276666
epoch 15 : loss 149.75363 ; accuracy 0.58376664
epoch 16 : loss 144.49825 ; accuracy 0.5937833
epoch 17 : loss 139.71014 ; accuracy 0.60285
epoch 18 : loss 135.32462 ; accuracy 0.6113833
epoch 19 : loss 131.30399 ; accuracy 0.61915
epoch 20 : loss 127.60723 ; accuracy 0.62596667
epoch 21 : loss 124.181786 